<a href="https://colab.research.google.com/github/gh-yana/testing1/blob/master/Handson_Let's_use_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# はじめに

![代替テキスト](https://scontent-itm1-1.xx.fbcdn.net/v/t31.0-8/s960x960/23117087_783917685113391_4777910378115406214_o.jpg?_nc_cat=111&_nc_sid=8024bb&_nc_ohc=AyIzXlIqzKAAX_jhHWA&_nc_ht=scontent-itm1-1.xx&_nc_tp=7&oh=e93af8553fb69966a386afd4d4939cd3&oe=5EE36A19)

# **BERT**簡易ハンズオン
### ～今、NLP界隈で熱い！BERTに触れてみる～

<font size=-1>**BERT**(**B**idirectional **E**ncoder **R**epresentations from **T**ransformers：トランスフォーマーによる双方向のエンコード表現)

BERTは、Googleが2018年10月に発表した自然言語処理(NLP)モデルで、略称の一部にもあるTransformerをベースとしている。「自然言語処理でもとうとう人間を超える精度を叩き出した」とまで言われるほど大きな話題にもなった。NLP界隈では「2019年はBERTとTransformerの年だった」とも言われており、近年のNLPの動向もBERTが中心となっている。

<font size=-1>（BERTに関する解説例は、一先ずこちらの[AINOW記事](https://ainow.ai/2019/05/21/167211/)をご参照。その他はBERTで検索してください。）

<font size=-1>（読みは、バート。バイディレクショナル・エンコーダー・リプレゼンテーション・フロム・トランスフォーマー）

**BERTの特長**

BERTは、自然言語処理タスクを**教師なし**でかつ双方向に事前学習する画期的なモデル。<b><font size=-2>
（教師なしデータでモデル作成できるため、普通のテキストコーパスのみを用いた訓練が可能。即ち、Web上の莫大な 普通のテキストデータが利用できることが重要なポイント。</b>

更に、モデルの**応用**ができる点もポイント。

*   <b><font size=-2>事前学習：pre-trainingにかなりのコストがかかる(Cloud TPUで4日～30日など)が、多くの事前学習済みモデルが予めリリースされており、それを使えば良い。
*   <b><font size=-2>転移学習：Fine-tuningは低コストで処理でき、Cloud TPUで1時間、GPUならば2時間程度で終わる。


## BERTの準備

In [0]:
!pip install transformers     # Transformersのインストール
!pip install mecab-python3    # MeCabのインストール
!pip install googletrans      # googletransインストール（後で使う）
import torch                  # パッケージのインポート
from transformers import pipeline,AutoTokenizer,BertTokenizer,AutoModelForSequenceClassification,BertJapaneseTokenizer, BertForMaskedLM  # パッケージのインポート

## Tokenizer
Tokenizerは、文章を単語ごとに分割(分かち書き)し、数字を割り当てます。

単語に分解し、その単語を**一意の数字**に置き換えることで、行列演算による機械学習を可能としている。



In [0]:
#@title 分かち書きしたい文章を以下に入力。 { run: "auto" }
tokenizer = BertJapaneseTokenizer.from_pretrained('daigo/bert-base-japanese-sentiment') #モデル読み込み
sentence = '今日の天気は晴れ。' #@param {type:"string"}
tokenized_sentence = tokenizer.tokenize(sentence,)
print(tokenized_sentence)
encoded_sentence = tokenizer.encode(sentence)
print(encoded_sentence)

<補足>　<font size=-1>※CLSは入力の先頭、SEPは分の区切りを表すトークン※

サンプル文章：**今日の天気は晴れ**

　↓（分かち書きの結果）↓
<table border="1" align="left">
<tr><th>[CLS]</th><th>今日</th><th>の</th><th>天気</th><th>は</th><th>晴れ</th><th>[SEP]</th></tr>
<tr><th>2</th><th>3246</th><th>5</th><th>11385</th><th>9</th><th>16577</th><th>3</th></tr>
</table>




---



# BERTを使ってみる
*   感情分析
*   単語推定
*   文章要約

※間に合わなかったが、その他に質問回答（質問すると回答を予測する）や文章作成（文章を与えると、勝手に色々文章を作ってくれる）など、世の中にはいろいろな物があります。


## 感情分析
文章がポジティブなのかネガティブなのかを判定（感情分析）する。

In [0]:
#@title 感情分析したい文章を以下に入力。 { run: "auto" }
sentence = '雨が降っていたが、午後から晴れた。' #@param {type:"string"}
tokenizer = AutoTokenizer.from_pretrained("daigo/bert-base-japanese-sentiment")  # 学習済みモデル読み込み
model = AutoModelForSequenceClassification.from_pretrained("daigo/bert-base-japanese-sentiment")  # 学習済みモデル読み込み

#TaskのParameterにsentiment-analysis(感情分析)を設定
print(pipeline("sentiment-analysis",model="daigo/bert-base-japanese-sentiment",tokenizer="daigo/bert-base-japanese-sentiment")(sentence))

#tokenized_sentence = tokenizer.tokenize(sentence)
#print(tokenized_sentence)
#encoded_sentence = tokenizer.encode(sentence)
#print(encoded_sentence)

<font size=-1>**＜補足＞**

<font size=-1>「雨が降っている」はネガティブだが、「午後から晴れた」加えるとポジティブに変わる。文脈を理解しながら判定していることが伺える。

## 単語推定
文章の一部をマスクして、マスクした部分の言葉を推定する。

In [0]:
tokenizer = BertJapaneseTokenizer.from_pretrained('bert-base-japanese-whole-word-masking')  # 学習済みモデルの読み込み
model = BertForMaskedLM.from_pretrained('bert-base-japanese-whole-word-masking')  # 学習済みモデルの読み込み

#TaskのParameterにfill-mask(単語推定)を設定
nlp = pipeline("fill-mask",model='bert-base-japanese-whole-word-masking',tokenizer='bert-base-japanese-whole-word-masking')
nlp(f"今日はテレビで{nlp.tokenizer.mask_token}の試合を見る。")
#nlp(f"学生時代はバレーボールをしていた。今日はテレビで{nlp.tokenizer.mask_token}の試合を見る。")

<font size=-1>**＜補足＞**

<font size=-1>「今日はテレビで[MASK]の試合をみる」の前に<font color=red>文章を付け足すことで、推定する言葉が変わってくる。<font color=black>このように、文脈を理解しながら推定していることが伺える。


##文章要約
文章を要約する。

<font size=-1><b>使うモデルが日本語対応していないため、日本語を英語翻訳した上で文章要約し、要約結果の英文を日本語翻訳する。<font color=red>（なんちゃって文書要約・・・）


In [0]:
sentence = 'ＦＢＩと国土安全保障省の共同声明によると、ＦＢＩは中国系ハッカー集団による米研究機関などへのサイバー攻撃の調査を進めている。これまでの調査で、ネットワークを通じ、ワクチンや治療法、検査に関わる知的財産や関連情報を特定し、不正に取得しようとする動きが確認された。' #@param {type:"string"}

from googletrans import Translator
translator = Translator()
trans_en = translator.translate(sentence)
print("-----要約モデルに流し込む英文を用意（googleで英訳）-----")
print(trans_en.text)

In [0]:
text = trans_en.text
#TaskのParameterにsummarization(要約)を設定
summarizer = pipeline("summarization",model="bart-large-cnn")
summary=summarizer(text, min_length=5, max_length=30)

In [0]:
print("-----要約結果の確認(取りあえず英文のまま)----")
data =json.dumps(summary[0]['summary_text'])
print(data)

In [0]:
summary_ja = translator.translate(data, dest='ja')
print("-----要約結果(英文)をgoogleで和訳----")
print(summary_ja.text)

<font size=-1>**＜補足＞**

<font size=-1>google translatorの精度が良い為、そこそこ使えるような気がする。
